In [1]:
import warnings
warnings.filterwarnings("ignore")

# import custom modules
from custom_libs import db
from custom_libs import preprocessing
from custom_libs import classification
from custom_libs import plotting
from custom_libs import utils
from custom_libs import dump
from custom_libs import db

In [2]:
text_column_name = 'review'
cleaned_text_column_name = 'review_cleaned'

In [3]:
df = db.get_dataset("McDonald_s_Reviews")
preprocessing.add_id_column(df, columns=["latitude", "longitude"])
preprocessing.add_rating_number_column(df)
proprocessing_function = preprocessing.preprocess_text
preprocessing.preprocess_dataframe(df, text_column_name, cleaned_text_column_name, proprocessing_function)
df.head(2)

Number of unique ids:  39


In [ ]:
sentiment_our_column_name = 'sentiment_our'
sentiment_auto_column_name = 'sentiment_auto'

In [ ]:
# Our model
model = dump.load_model("model_chat")
vectorizer = dump.load_model("vectorizer_chat")
df[sentiment_our_column_name] = model.predict(vectorizer.transform(df[cleaned_text_column_name]))

# SentimentIntensityAnalyzer model
classification.append_sentiment_for_each_row(df, cleaned_text_column_name, new_column_name=sentiment_auto_column_name)

# Results
df[[cleaned_text_column_name, sentiment_auto_column_name, sentiment_our_column_name]]

In [ ]:
# Rows with different sentiment
df_differences = df[df[sentiment_our_column_name] != df[sentiment_auto_column_name]]
df_differences.shape

In [ ]:
df_differences = df_differences[[text_column_name, sentiment_auto_column_name, sentiment_our_column_name]]
db.save_dataset(df_differences, "McDonald_s_Reviews_differences")

In [ ]:
# df[[cleaned_text_column_name, sentiment_auto_column_name, sentiment_our_column_name]].sample(10)

In [ ]:
def plot_sentiment_word_clouds(df):
    for sentiment in classification.Sentiment.get_all():
        df_sentiment = df[(df[sentiment_auto_column_name]==sentiment)]
        plotting.plot_word_cloud(df_sentiment, sentiment, cleaned_text_column_name, plt_name="Auto")
        df_sentiment = df[(df[sentiment_our_column_name]==sentiment)]
        plotting.plot_word_cloud(df_sentiment, sentiment, cleaned_text_column_name, plt_name="Our")

# plot_sentiment_word_clouds(df)

In [ ]:
current_position = [30.460718, -97.792874]
max_distance = 20
best_rated_restaurant = utils.select_best_restaurant_from_stars(df, current_position, max_distance)
best_clf_column = sentiment_auto_column_name
best_feeling_restaurant = utils.select_best_restaurant_from_sentiment(df, current_position, max_distance, sentiment_column=best_clf_column)
best_rated_restaurant, best_feeling_restaurant